In [ ]:
#Installing and importing needed libraries
import os
import skextremes as ske
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from io import StringIO

!pip install git+https://github.com/OpenHydrology/lmoments3.git

!git clone https://github.com/kikocorreoso/scikit-extremes.git

os.chdir('/content/scikit-extremes')

!cd /content/scikit-extremes
!pwd

!pip install -e .


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# The path of different datasets, including bankfull_width, Po river simulated discharges, and observed discharches

bankfull_width_dir='/content/drive/MyDrive/bankfull_width.nc'
discharge_dir='/content/drive/MyDrive/discharge_dailyTot_output.nc'
databaseQmax_Po_dir='/content/drive/MyDrive/databaseQmax_Po.dat'


In [ ]:
# Creating different dataframes 
ds1=xr.open_dataset(discharge_dir)
df1=ds1.to_dataframe()

ds2=xr.open_dataset(bankfull_width_dir)
df2=ds2.to_dataframe()



with open(databaseQmax_Po_dir, 'r') as reading:
    my_string = reading.read()
buffer = StringIO(my_string)
qmax_observations = pd.read_csv(buffer, sep=" ")

# Selecting desired columns(or desired stream gauges) and the desired interval for later comparison (2000-2010)
qmax_observations=qmax_observations[qmax_observations['codice'].isin(['B102' ,'B097' ,'B052','B047'])][['codice' ,'a','QC']]
qmax_observations_2000_2010=qmax_observations[qmax_observations['a'].between(2000,2010)]
qmax_observations_2000_2010=qmax_observations_2000_2010.pivot_table(index='a' , columns='codice' , values='QC')
qmax_observations_2000_2010.index

# Renaming the desired columns to the real stream gauge names
qmax_observations_2000_2010.rename(columns={'B047' : 'CREMONA' , 'B052' : 'FICAROLO' , 'B097': 'SERMIDE' , 'B102' : 'SPESSA'} , inplace=True)


df1 = df1.reset_index()
df1 = df1.pivot_table(index=['lat' , 'lon'], 
                     columns=pd.Grouper(freq='A', key='time'),  
                     aggfunc='max', 
                     values='discharge',
                     fill_value=0)
# Joining the simulated dischrges and the bankfull width to extract the cells with bankful greater than 50
df3=df1.join(df2)
river_mask=df3.loc[lambda df: df['bankfull_width_map'] > 50, :]
river_mask.drop('bankfull_width_map' , axis=1 , inplace=True)

# Tuning(re-sacaling) the simulated discharge for each cell relative to the real contributing area
for col in river_mask.columns :
  river_mask[col]=river_mask[col]*2.15

# Selecting the desired cells for comparison with the existing stream gauges
selected_cells_list=[(45.125 , 9.875) , (44.95833206176758,11.375) ,(45.04166793823242, 11.125),  (45.125 , 9.375) ]
selected_cells=river_mask.loc[selected_cells_list]
b=selected_cells.T

# Plotting the observed discharges and simulated ones 
for i in range(0,4) :
   fig = plt.figure()
   plt.plot(b.index , qmax_observations_2000_2010[qmax_observations_2000_2010.columns[i]] , label=qmax_observations_2000_2010.columns[i])
   plt.plot(b[b.columns[i]] , label= b.columns[i])
   plt.legend()
   plt.xlabel('year')
   plt.ylabel('discharges(m3/s)')
   plt.title('Annual maximum discharge')
plt.show()


river_mask = pd.DataFrame(np.sort(river_mask.to_numpy(), axis=1), river_mask.index, river_mask.columns)


In [ ]:
# Defining the Gumble function
def Gumbel(sample):
    """ Gumbel distribution funcrion :
    Parameters:
    sample : array_like
    1D array_like with the extreme values to be considered
    Returns:
    location-parameter : float 
    scale-paramete : float 
    q100 : float
    """

    model=ske.models.classic.Gumbel(sample , fit_method='mle' , ci=0.05 , ci_method='delta')
    q100=model.ppf(0.99)
    #model.plot_summary()

    return  q100


In [ ]:
# Applying the gumble function for each simulated cell to compute 100-year flood
river_mask['q100(m^3/s)']=river_mask.apply( lambda x : Gumbel(x) , axis=1 )


In [ ]:
# Changing the column names to the representing years
for i in range(0,len(river_mask.columns.values)-1) :
    pd.to_datetime(river_mask.columns.values[i])
    river_mask.columns.values[i]=river_mask.columns.values[i].year

# Saving the output in csv format
river_mask.to_csv('/content/drive/MyDrive/Annual-Maxima-discharges-rescale.csv')
